# Imports and Utility Functions

In [25]:
import codecs


def repeat_key_xor(message, key):
    n, m = len(message), len(key)
    result = []
    for i in range(n):
        byte = message[i] ^ key[i % m]
        result.append(byte)
    return bytes(result)


LETTERS = 'etaoinsrhldcumfpgwybvkxjqz'
SCORES = [x for x in range(len(LETTERS), 0, -1)]
def text_score(S):
    """
    Returns the score of a piece of text looking only at letter 
    frequency.
    """
    score = 0
    for i in S:
        ch = chr(i)
        if ch != '\n' and not ch.isprintable():
            return -1
        if ch in LETTERS:
            score += SCORES[LETTERS.index(ch)]
    return score


def popcount(x):
    return bin(x).count('1')


def hamming_distance(A, B):
    """
    (bytes, bytes) -> int
    Returns the hamming distance between 'bytes' A and B.
    """
    assert len(A) == len(B), 'lengths are not equal.'

    distance = sum(popcount(x ^ y) for x, y in zip(A, B))
    return distance


## Challenge 1.1
Convert hex to base64

In [26]:
hex_string = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
b = codecs.decode(hex_string, 'hex')
base64_string = codecs.encode(b, 'base64')

base64_string

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t\n'

## Challenge 1.2: Fixed XOR
Write a function that takes two equal-length buffers and produces their XOR combination.

In [27]:
hex1 = '1c0111001f010100061a024b53535009181c'
hex2 = '686974207468652062756c6c277320657965'
b1 = codecs.decode(hex1, 'hex')
b2 = codecs.decode(hex2, 'hex')

result = [x ^ y for x, y in zip(b1, b2)]
bytes(result).hex()

'746865206b696420646f6e277420706c6179'

## Challenge 1.3: Single-byte XOR cipher

In [28]:
def single_key_xor(b):
    best = (-1, b'')
    for guess in range(256):
        b1 = repeat_key_xor(b, bytes([guess]))
        score = text_score(b1)
        best = max(best, (score, b1))
    return best

hex1 = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
b1 = codecs.decode(hex1, 'hex')
    
single_key_xor(b1)

(428, b"Cooking MC's like a pound of bacon")

## Challenge 1.4: Detect single-character XOR

In [35]:
best = (-1, b'')
with open('data/4.txt', 'r') as fin:
    for line in fin:
        ans = single_key_xor(codecs.decode(line.rstrip(), 'hex'))
        best = max(best, ans)
best

(416, b'Now that the party is jumping\n')

## Challenge 1.5: Implement repeating-key XOR

In [30]:
msg = '''Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal'''

key = 'ICE'
b = repeat_key_xor(bytes(ord(x) for x in msg), bytes(ord(x) for x in key))

codecs.encode(b, 'hex')

b'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

## Challenge 1.6: Break repeating-key XOR

In [31]:
s1 = 'this is a test'
s2 = 'wokka wokka!!!'

hamming_distance(s1.encode('utf-8'), s2.encode('utf-8'))

37

In [34]:
msg = codecs.decode(open('data/6.txt', 'r').read().encode('utf-8'), 'base64')
n = len(msg)
keysizes = []
for k in range(2, 41):
    total_distance = 0
    for i in range(0, n, k):
        if i + 2*k >= n:
            break
        total_distance += hamming_distance(msg[i:i + k], msg[i + k:i + 2*k])
    keysizes.append((k, total_distance))

keysizes.sort(key=lambda x: x[1])
for k in keysizes[:1]:
    size = k[0]
    blocks = [[] for _ in range(size)]
    for i in range(n):
        blocks[i % size].append(msg[i])

    best = []
    for block in blocks:
        best.append(single_key_xor(bytes(block))[1])

    reconstructed = []
    for i in range(n):
        reconstructed.append(best[i % size][i // size])
        
    print(bytes(reconstructed).decode('utf-8'))


I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 
